# This is a jupyter notebook that outlines the child Diet Quality Index
grace.shearrer@gmail.com

In [1]:
import os, glob
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from zipfile import ZipFile
import argparse
import pdb

In [2]:
# function to read nonlists easier
def PP(nonlist):
    print(list(nonlist))
    return(nonlist)

## These are dictionaries and lists that contain the parameters for the data set
* Important -> the variables of interest
* units -> to caluclate the DQI one needs to convert the data to the same units this dictionary has the conversions
* ped811_dict_> This and the other ped%i%i_dict have the serving paramerers the convention is name of the variable as the key, the values are the parameters (low and high) and the name of the score variable. The score variable has an X in it
* ped_dict -> This dictionary is relating the above with the more human friendly words
* hei_ped_dict -> This dictionary has the food groupings and the NDSR variable names associated with each group it also has the HEI groupings and the generated variables associated with each
* ped_interest -> This has the specific interest variables for the pediatric variables
* demo_interest -> The demographic variables of interest
* BF_dict -> This is the dictionary that relates the variable names from qualtrics to easier variable names
* calc_dict -> This is a dictonary that subsets the data by age into the 3 data frames (infant, young, child)
* HEI_cols -> This is a dictionary that has all the variables that contain SCORES (HEIX) and will be summed to produce the final HEI score
* bf_dict -> This dictionary will have keys (infant, young, child) the values are the dataset for each key group with the final HEI calculations

In [3]:
#    Dictionaries and lists
important=['Participant ID','Date of Intake',
     'Whole Grains (ounce equivalents)','Sodium (mg)','Refined Grains (ounce equivalents)',
     'Added Sugars (by Total Sugars) (g)','% Calories from SFA','Energy (kcal)',
     'Total Polyunsaturated Fatty Acids (PUFA) (g)','Total Monounsaturated Fatty Acids (MUFA) (g)',
     'Total Saturated Fatty Acids (SFA) (g)',
     'FRU0100','FRU0200','FRU0300','FRU0400','FRU0500','FRU0600','FRU0700',
     'VEG0100','VEG0200','VEG0300','VEG0400','VEG0800','VEG0450','VEG0700',
     'VEG0600','VEG0900','GRW1100','GRS0100','GRR0100','GRW0500','GRS0500',
     'MSC0900','MSC1000','VEG0500','DOT0700','DOT0800','BVS0400','BVA0400',
     'BVU0300','BVS0300','BVA0300','BVS0500','BVA0500', 'BVU0400','BVS0100', 
     'BVA0100', 'BVU0100', 'BVS0200','BVA0200','BVU0200','BVS0600', 'BVA0600',
     'BVU0500', 'BVS0700', 'BVA0700', 'BVU0600','GRW0400','GRS0400','GRR0400','GRW0600',
     'GRS0600','GRR0600','GRW0700','GRW0700','GRS0700','GRR0700','GRW0900','GRS0900',
     'GRR0900','MRF0100','MRL0100','MRF0200','MRL0200','MRF0300','MRL0300','MRF0400',
     'MRL0400','MCF0200','MCL0200','MRF0500','MPF0100','MPL0100','MPF0200','MFF0200',
     'MSL0100','MSF0100','MCF0100','MCL0100','MOF0100','MOF0200','MOF0700','FMC0100',
     'FMC0200','MOF0500', 'MOF0600','FCF0100','SWT0700','SWT0800','GRW0200','GRS0200','GRR0200',
     'GRW0300','GRS0300','GRR0300','SWT0300','GRW0800','GRS0800','GRR0800','GRW1000','GRS1000',
     'GRR1000','SWT0100','SWT0200','SWT0400','DMF0100','DMR0100','DML0100','DMN0100','DMF0200',
     'DMR0200','DML0200',
     'DML0300','DML0400','DYF0100', 'DYR0100','DYL0100','DYR0200',
     'DYL0200','DYF0200', 'DYN0100',
     'DYN0100','DOT0300','DOT0400', 'DOT0500', 'DOT0600',
     'SWT0600','MOF0300','MOF0400','DCF0100','DCR0100','DCL0100','DCN0100','DOT0200', 'MSC0600', 'SWT0500', 'DOT0100'
     ]

units={  
  'keep_serv':['SWT0100','SWT0200','SWT0300','SWT0700' , 'SWT0800','SWT0400','SWT0500','DOT0300' , 'DOT0400','DOT0100',
               'DOT0200','GRR0800' , 'GRS0800',  'GRW0800','MSC0900','MSC0600'],
  'keep_04':['Whole Grains (ounce equivalents)','Sodium (mg)','Refined Grains (ounce equivalents)',
     'Added Sugars (by Total Sugars) (g)','% Calories from SFA','Energy (kcal)',
     'Total Polyunsaturated Fatty Acids (PUFA) (g)','Total Monounsaturated Fatty Acids (MUFA) (g)',
     'Total Saturated Fatty Acids (SFA) (g)'],
  'cup':{
    '.5':
      ['FRU0300','FRU0400','FRU0500','FRU0600','FRU0700',
       'VEG0100','VEG0200','VEG0300','VEG0400','VEG0800','VEG0450','VEG0700',
       'VEG0600','VEG0900','GRW1100','GRS0100','GRR0100','GRW0500','GRS0500',
       'MSC1000'],
    '1':
      ['DMF0100','DMR0100','DML0100','DMN0100','DMF0200','DMR0200','DML0200',
       'DML0300','DML0400','DYF0100', 'DYR0100','DYL0100','DYR0200',
       'DYL0200','DYF0200', 'DYN0100',
       'DYN0100','DOT0500', 'DOT0600',
       'SWT0600']
       },
  'floz':{
    '4':
      ['VEG0500','FRU0100','FRU0200'],
    '5':
      ['DOT0700','DOT0800'],
    '8':
      ['BVS0400','BVA0400','BVU0300','BVS0300','BVA0300','BVS0500','BVA0500', 'BVU0400',
       'BVS0100', 'BVA0100', 'BVU0100', 'BVS0200','BVA0200','BVU0200','BVS0600', 'BVA0600',
       'BVU0500', 'BVS0700', 'BVA0700', 'BVU0600']
  },
  'oz':{
    '.5':
        ['MOF0500'],
    '1':
      ['GRW0400','GRS0400','GRR0400','GRW0600',
         'GRS0600','GRR0600','GRW0700','GRW0700','GRS0700','GRR0700','GRW0900','GRS0900',
         'GRR0900','MRF0100','MRL0100','MRF0200','MRL0200','MRF0300','MRL0300','MRF0400',
         'MRL0400','MCF0200','MCL0200','MRF0500','MPF0100','MPL0100','MPF0200','MFF0200',
         'MSL0100','MSF0100','MCF0100','MCL0100','MOF0100','MOF0200','MOF0700','FMC0100',
         'FMC0200'],
    '2':
      ['DCF0100','DCR0100','DCL0100','DCN0100'],
    '28':
      ['GRW0200','GRS0200','GRR0200'],
    '45':
      ['GRW0300','GRS0300','GRR0300']
  },
  'TB':{
    '1':
      ['MOF0600','FCF0100']
  },
  'g':{
    '40':
      ['GRW1000','GRS1000','GRR1000']
  },
  'egg':{
    '1':
      ['MOF0300','MOF0400']
  }
}      

           
ped811_dict = {
    'HEI_TOTALVEG': {'parameters':[0.1,1.9], 'name': 'HEIX1_TOTALVEGETABLES'},
    'HEI_TOTALFRUIT': {'parameters':[0.1,1.9], 'name': 'HEIX2_TOTALFRUIT'},
    'HEI_WHOLEGRAINS': {'parameters':[1.0,3.5,0.0 , 8.0], 'name': 'HEIX3_WHOLEGRAIN'},
    'HEI_MILK': {'parameters':[20,28,8,35], 'name': 'HEIX4_MILK'},
    'HEI_TOTALPROTEINS': {'parameters':[2.5,6.0,0.0,10.0], 'name': 'HEIX5_TOTALPROTEIN'},
    'HEI_REFINEDGRAINS': {'parameters':[1.6,3.5], 'name': 'HEIX6_REFINEDGRAIN'},
    'HEI_FRUITJUICE': {'parameters':[0.1,6.0], 'name': 'HEIX7_FRUITJUICE'},
    'HEI_SSB': {'parameters':[0.1,4.0], 'name': 'HEIX8_SSB'},
    'HEI_SWEETS': {'parameters':[0.1,1.0], 'name': 'HEIX9_SWEETS'},
    'HEI_SALTY': {'parameters':[0.1,1.0], 'name': 'HEIX10_SALTY'}
    }

ped1224_dict = {
    'HEI_TOTALVEG': {'parameters':[0.1,7.9], 'name': 'HEIX1_TOTALVEGETABLES'},
    'HEI_TOTALFRUIT': {'parameters':[0.1,7.9], 'name': 'HEIX2_TOTALFRUIT'},
    'HEI_WHOLEGRAINS': {'parameters':[1.5, 5.5, 0, 8.0], 'name': 'HEIX3_WHOLEGRAIN'},
    'HEI_MILK': {'parameters':[14.0,18.0,8.0,24.0], 'name': 'HEIX4_MILK'},
    'HEI_TOTALPROTEINS': {'parameters':[2.0,3.0,0,6.0], 'name': 'HEIX5_TOTALPROTEIN'},
    'HEI_REFINEDGRAINS': {'parameters':[1.9, 4.2], 'name': 'HEIX6_REFINEDGRAIN'},
    'HEI_FRUITJUICE': {'parameters':[4.1,6.0], 'name': 'HEIX7_FRUITJUICE'},
    'HEI_SSB': {'parameters':[0.1,4.0], 'name': 'HEIX8_SSB'},
    'HEI_SWEETS': {'parameters':[0.1,1.0], 'name': 'HEIX9_SWEETS'},
    'HEI_SALTY': {'parameters':[0.1,1.0], 'name': 'HEIX10_SALTY'}
    }

ped08_dict = {
    'HEI_TOTALVEG': {'parameters':[0,0], 'name': 'HEIX1_TOTALVEGETABLES'},
    'HEI_TOTALFRUIT': {'parameters':[0,0], 'name': 'HEIX2_TOTALFRUIT'},
    'HEI_WHOLEGRAINS': {'parameters':[0,0,0,0], 'name': 'HEIX3_WHOLEGRAIN'},
    'HEI_MILK': {'parameters':[0,0,0,0], 'name': 'HEIX4_MILK'},
    'HEI_TOTALPROTEINS': {'parameters':[0,0,0,0], 'name': 'HEIX5_TOTALPROTEIN'},
    'HEI_REFINEDGRAINS': {'parameters':[0,0], 'name': 'HEIX6_REFINEDGRAIN'},
    'HEI_FRUITJUICE': {'parameters':[0,0], 'name': 'HEIX7_FRUITJUICE'},
    'HEI_SSB': {'parameters':[0,0], 'name': 'HEIX8_SSB'},
    'HEI_SWEETS': {'parameters':[0,0], 'name': 'HEIX9_SWEETS'},
    'HEI_SALTY': {'parameters':[0,0], 'name': 'HEIX10_SALTY'},
    'HEI_CEREAL': {'parameters':[0,0], 'name': 'HEIX11_CEREAL'}
    }

ped_dict={
    'young': ped811_dict,
    'child':ped1224_dict,
    'infant':ped08_dict
    }

hei_ped_dict={
    'hei':{
        'hei_sweets':['SWT0100','SWT0200','SWT0300','SWT0700' , 'SWT0800','SWT0400',
                     'SWT0500','DOT0300' , 'DOT0400','DOT0100','DOT0200','GRR0800' ,
                     'GRS0800', 'GRW0800','MSC0900','MSC0600'],
        'hei_salty':['GRW0900' ,'GRS0900', 'GRR0900', 'GRW1100', 'GRW0400',
                     'GRS0400' ,'GRR0400','VEG0800','FMC0100', 'FMC0200'],
        'hei_veg': ['VEG0100','VEG0200','VEG0300','VEG0400','VEG0800','VEG0450','VEG0700','VEG0600','VEG0900','VEG0500'],
        'hei_greensbeans': ['VEG0100','VEG0700'],
        'hei_totalfruit': ['FRU0100','FRU0200','FRU0300','FRU0400','FRU0500','FRU0600','FRU0700'],
        'hei_wholegrains': ['Whole Grains (ounce equivalents)'],
        'hei_refinedgrains': ['Refined Grains (ounce equivalents)'],
        'hei_proteins': ['MRF0100','MRL0100','MRF0200','MRL0200','MRF0300','MRL0300','MRF0400',
                        'MRL0400','MCF0200','MCL0200','MRF0500','MPF0100','MPL0100','MPF0200',
                        'MFF0200','MSL0100',
                        'MSF0100','MCF0100','MCL0100','MOF0100','MOF0200','MOF0300','MOF0400','MOF0500',
                        'MOF0600','MOF0700','VEG0700'],
        'hei_seafoodplantprot':['MFF0200','MSL0100','MSF0100','MOF0500','MOF0600','MOF0700','VEG0700'],
        'hei_dairy': ['DMF0100','DMR0100','DML0100','DMN0100','DMF0200','DMR0200','DML0200','DML0300','DML0400','DCF0100','DCR0100','DCL0100','DCN0100','DYF0100',
                      'DYR0100','DYL0100','DYF0200','DYR0200','DYL0200','DYN0100','DOT0300','DOT0400','DOT0500','DOT0600','DOT0100'],
        'hei_dairyprot': ['DCF0100','DCR0100','DCL0100','DCN0100','DCN0100','DYF0100',
                          'DYR0100','DYL0100','DYF0200','DYR0200','DYL0200','DYN0100'],
        'hei_milk': ['DMF0100','DMR0100','DML0100','DMN0100' ,'DMF0200','DMR0200','DML0200','DOT0500', 'DOT0600'],
        'hei_formula_foz': ['DOT0700', 'DOT0800'],
        'hei_cereal': ['GRW0600','GRS0600','GRR0600','GRW0700','GRS0700','GRR0700'],
        'hei_fruitjuice': ['FRU0100','FRU0200'],
        'hei_ssb': ['DMF0200','DMR0200','DML0200','DML0300','SWT0600','BVS0400','BVS0300','BVS0500',
                    'BVS0100','BVS0200','BVS0600','BVS0700', 'SWT0600'],
        'ENERGY': ['Energy (kcal)']
    },
    'HEI':{
        'HEI_TOTALVEG': ['hei_veg', 'hei_greensbeans'],
        'HEI_TOTALPROTEINS': ['hei_proteins','hei_seafoodplantprot','hei_dairyprot'],
        'HEI_MILK': ['hei_milk','hei_formula_foz'],
        
        'HEI_TOTALFRUIT': ['hei_totalfruit'],
        'HEI_WHOLEGRAINS': ['hei_wholegrains'],
        
        'HEI_REFINEDGRAINS': ['hei_refinedgrains'],
        'HEI_FRUITJUICE': ['hei_fruitjuice'],
        'HEI_SSB': ['hei_ssb'],
        'HEI_SWEETS': ['hei_sweets'],
        'HEI_SALTY': ['hei_salty'],
        'HEI_CEREAL': ['hei_cereal']
    }
}



ped_interest = ['Participant ID','Energy (kcal)', 'hei_totveg', 'hei_greensbeans', 'hei_totfruit', 'hei_wholefruit', 'hei_wholegrains','hei_dairy', 'hei_totproteins', 'hei_seafoodplantprot',
            'hei_refinedgrains', 'hei_addedsugars', 'ripctsfa','energy','chocolate_candies','candies','frosting','sweet_sauce','sugar','syrups','Pudding', 'icecream','nondairy_treat','baked_good',
            'chips','other_fried','fries']
demo_interest = ["Identifiers_visit", "Date_taken","weight","head_circumference","height","Participant ID","Gender","DoB"]

BF_dict = {'Participant ID' : 'CandID',
           'breastfed': '[Was the child ever breastfed or fed breast milk?] [y/n]', 
           'age_stop': '[How old was the child when s/he completely stopped breastfeeding or being fed breastmilk? ][months]',
           'age_any_formula': '] [months]',
           'age_regular_formula': '[How old was the child when s/he was first regularly fed formula (regularly/routinely given as part of their diet, not as part of a one-time medical intervention, etc.)?] [months]',
           'age_stop_formula':'[How old was the child when s/he completely stopped being fed formula? [months]',
           'age_fed': '[How old was the child when s/he was first fed anything other than breast milk, formula, or water (ex: rice cereal, pureed vegetables, etc.)?] [months]',           
           'donor_milk': '[Was your child given donor breastmilk?] [y/n]',
           'donor_milk_percent':'[If yes, what % of all breastmilk given to your child was donor breastmilk] [%]',
           'four_month_ratio':'[For the first 4 months, was your child]'}

# Listed here but actually doesn't work till later          
#calc_dict={
#    'infant':HEI_components(ped_dict['infant'],df.loc[df['age at intake'] <= 6]),
#    'young':HEI_components(ped_dict['young'],df.loc[(df['age at intake'] > 6) & (df['age at intake'] <= 11)]),
#    'child':HEI_components(ped_dict['child'],df.loc[(df['age at intake'] > 11 ) & (df['age at intake'] <= 24)])}

HEI_cols = ['HEIX0_BF','HEIX1_TOTALVEGETABLES', 
            'HEIX2_TOTALFRUIT', 'HEIX3_WHOLEGRAIN', 
            'HEIX4_MILK', 'HEIX5_TOTALPROTEIN', 
            'HEIX6_REFINEDGRAIN', 'HEIX7_FRUITJUICE', 
            'HEIX8_SSB', 'HEIX9_SWEETS', 'HEIX10_SALTY']
# listed here but not produced till later 
# bf_dict = {}


## RECOMEND setting the basepath to your basepath
arglist is here as I am preparing to take this out of a notebook and into a python module, just keep as is

In [4]:
basepath='YOUR PATH HERE'
arglist={'BASEPATH': basepath, 
         'CHILD': True, 
         'NAMES': 'test', 
         'OPTS': ['Chil'], 
         'SAVE':os.path.join(basepath, 'derivived'),
         'XTRA': 'total_demo_dataset.csv'}

## Finding the data from zipped files

In [5]:
# Find the data
for (dirpath, dirnames, filenames) in os.walk(basepath):
   for filename in filenames:
       if filename.endswith('.zip'):
           tmppath=os.sep.join([dirpath, filename])
           with ZipFile(tmppath, 'r') as zipObj:
              # Get a list of all archived file names from the zip
              listOfFileNames = zipObj.namelist()
              # Iterate over the file names
              for fileName in listOfFileNames:
                  # Check filename endswith txt
                   if fileName.endswith('04.txt'):
                       zipObj.extract(fileName, os.path.join(basepath,'temp_txt'))
                   if fileName.endswith('09.txt'):
                       zipObj.extract(fileName, os.path.join(basepath,'temp_txt'))
infile = os.path.join(basepath,'temp_txt')


### Function: ager -> calculates the child's age in months

In [6]:
def ager(date1,date2):
    # Get the age in months of each recall
    diff_time = pd.to_datetime(date1)-pd.to_datetime(date2)
    diff_time= diff_time/ np.timedelta64(1, 'M')
    return(diff_time)


### Function: file_org -> this is going to look at all the files from NDSR and find only the child related (or mom related) the output is a dictionary with the paths 

In [7]:
def file_org(infile, arglist):
    # will create a dictionary with the file paths to all the data
    if arglist['OPTS'] == False:
        arglist['OPTS']=[arglist['NAMES']]
        file_dict = {"set_04": {"%s"%arglist['NAMES']:{}}, "set_09": {"%s"%arglist['NAMES']: {}}}
        file_dict['set_04']["%s"%arglist['NAMES']]["files"] = [x for x in glob.glob(os.path.join(infile,'*04.txt')) if "%s"%arglist['NAMES'] in x]
        file_dict['set_09']["%s"%arglist['NAMES']]["files"] = [x for x in glob.glob(os.path.join(infile,'*09.txt')) if "%s"%arglist['NAMES'] in x]
        arglist['OPTS']=[arglist['NAMES']]

    else:
        file_dict={"set_04":{},"set_09":{}}
        for item in arglist['OPTS']:
            print('this is the item %s'%item)
            file_dict["set_04"][item]= {}
            file_dict["set_09"][item]= {}

        for key,value in file_dict.items():
            keys=list(file_dict[key].keys())
            for k in keys:
                print(k)
                file_dict['set_04']["%s"%k]["files"] = [x for x in glob.glob(os.path.join(infile,'*04.txt')) if '%s'%k in x]
                file_dict['set_09']["%s"%k]["files"] = [x for x in glob.glob(os.path.join(infile,'*09.txt')) if '%s'%k in x]

    return(file_dict)


In [8]:
a=file_org(infile, arglist) #list of the files


this is the item Chil
Chil
Chil


## Read in the demographics file
Rename the file so that the variables are easier to understand

In [9]:
df_bf=pd.read_csv(os.path.join(basepath,'2020-07-28_BCP-Enriched_Data Request_ChildFeedingPractice_final.csv'), sep=",")


In [10]:
test = dict(zip(df_bf.columns, BF_dict.keys()))

In [11]:
df_bf.rename(test, axis=1, inplace=True)

In [12]:
df_bf[['age_stop','age_any_formula', 'age_regular_formula','age_stop_formula','age_fed']] = df_bf[['age_stop','age_any_formula', 'age_regular_formula','age_stop_formula','age_fed']].apply(pd.to_numeric)

In [13]:
df_bf.dtypes

Participant ID           int64
breastfed               object
age_stop               float64
age_any_formula        float64
age_regular_formula    float64
age_stop_formula       float64
age_fed                float64
donor_milk              object
donor_milk_percent     float64
four_month_ratio        object
dtype: object

## Read in the files needed and put them in a dictionary

In [14]:
# proto file reader
data_dict={"set_04":{},"set_09":{},"demo":{}}
df_demo=pd.read_csv(os.path.join(basepath,arglist['XTRA']), sep=",")

df_demo['Participant ID'] = df_demo['Participant ID'].astype('int32')
df_demo['Identifiers_visit']=df_demo['Identifiers_visit'].astype('int32')

data_dict["demo"]= df_demo[demo_interest]
dob=data_dict['demo'][['DoB','Participant ID']].drop_duplicates(keep='last') 


In [15]:
for key, value in a.items():
    print(key)
    for k,v, in value.items():
        print(k)
        temp_list = []
        for file in v["files"]:
            print(file)
            temp_df =  pd.read_csv(file, sep="\t", encoding='latin1')
            if key == 'set_09':
                temp_df=temp_df.drop([0]) #drops extra row 
            for val in temp_df["Participant ID"]:
                _id = str(val).lstrip("0").split("_")[0]
                temp_df.replace(val, _id, inplace=True)
            temp_list.append(temp_df)
        dfm_original = pd.concat(temp_list, ignore_index=True)
        print("Final dataframe size: ", dfm_original.shape)
        dfm_original = dfm_original.sort_values(by="Participant ID")
         
        del_cols=set(dfm_original.columns) - set(important)
        dfm_original.drop(del_cols, axis=1, inplace=True)
        for col in dfm_original:
            if dfm_original[col].dtype == np.object_:
                dfm_original[col] = (dfm_original[col].replace(',','.', regex=True))   
        
        concat_filepath = os.path.join(arglist['SAVE'],'%s_dataset_%s.csv'%(k,key))

        dfm_original['Participant ID'] = dfm_original['Participant ID'].astype('float').astype('int32')
        dfm_original.sort_values("Participant ID", inplace=True) 

        dfm_original=dfm_original.drop_duplicates(keep='first') 
        
       
        dfm_original=pd.merge(dob, dfm_original,on='Participant ID')
        dfm_original['age at intake']=ager(dfm_original['Date of Intake'], dfm_original['DoB'])
        
        data_dict[key]= dfm_original


set_04
Chil
/Users/gracer/Google Drive/BCP/data/temp_txt/UNCChildJuly04.txt
/Users/gracer/Google Drive/BCP/data/temp_txt/UMNUNCChild04.txt
/Users/gracer/Google Drive/BCP/data/temp_txt/UMNChilMar1804.txt
/Users/gracer/Google Drive/BCP/data/temp_txt/ChildOct1804.txt
/Users/gracer/Google Drive/BCP/data/temp_txt/BCPChildMSTR04.txt
/Users/gracer/Google Drive/BCP/data/temp_txt/UMNChildJuly04.txt
/Users/gracer/Google Drive/BCP/data/temp_txt/BCPChildAP04.txt
Final dataframe size:  (938, 226)
set_09
Chil
/Users/gracer/Google Drive/BCP/data/temp_txt/BCPChildAP09.txt
/Users/gracer/Google Drive/BCP/data/temp_txt/UMNChildJuly09.txt
/Users/gracer/Google Drive/BCP/data/temp_txt/BCPChildMSTR09.txt
/Users/gracer/Google Drive/BCP/data/temp_txt/ChildOct1809.txt
/Users/gracer/Google Drive/BCP/data/temp_txt/UMNChilMar1809.txt
/Users/gracer/Google Drive/BCP/data/temp_txt/UMNUNCChild09.txt
/Users/gracer/Google Drive/BCP/data/temp_txt/UNCChildJuly09.txt
Final dataframe size:  (938, 171)


### Sort and check the data

In [16]:
data_dict['set_09'].sort_values("Participant ID", inplace=True) 
data_dict['set_04'].sort_values("Participant ID", inplace=True) 

In [17]:
data_dict['set_09']

DoB  Participant ID Date of Intake FRU0100 FRU0200 FRU0300 FRU0400  \
499   7/19/17            2081     08/12/2017       0       0       0       0   
500   7/19/17            2081     10/25/2018       0       0       0       0   
501   7/19/17            2081     08/03/2017       0       0       0       0   
548   2/14/17           11228     02/13/2018       0       0       0       0   
547   2/14/17           11228     09/09/2018       0       0       0       0   
..        ...             ...            ...     ...     ...     ...     ...   
205   4/25/17          993015     07/22/2018       0       0       0   1.381   
206   4/25/17          993015     01/05/2018       0       0       0   0.750   
199   4/25/17          993015     07/19/2017       0       0       0       0   
600  12/11/16          999647     04/13/2018       0       0       0   2.627   
601  12/11/16          999647     04/18/2018       0       0       0   1.729   

    FRU0500 FRU0600 FRU0700  ... BVS0600 BVA0600 BVU0500 BVS0700 BVA0700  \
499       0       0       0  ...       0       0   3.000       0       0   
500       0       0       0  ...       0       0       0       0       0   
501       0       0       0  ...       0       0       0       0       0   
548       0       0       0  ...       0       0   1.500       0       0   
547       0       0       0  ...       0       0   1.000       0       0   
..      ...     ...     ...  ...     ...     ...     ...     ...     ...   
205       0       0       0  ...       0       0       0       0       0   
206       0       0       0  ...       0       0       0       0       0   
199       0       0       0  ...       0       0       0       0       0   
600       0       0       0  ...       0       0       0       0       0   
601       0       0       0  ...       0       0   0.500       0       0   

    BVU0600 MSC0600 MSC0900 MSC1000 age at intake  
499       0       0       0       0      0.788517  
500       0       0       0       0     15.211811  
501       0       0       0       0      0.492823  
548       0       0       0       0     11.959178  
547       0       0       0       0     18.792994  
..      ...     ...     ...     ...           ...  
205       0       0       0       0     14.883262  
206       0       0       0   0.388      8.377995  
199       0       0       0       0      2.792665  
600       0       0       0       0     16.033183  
601       0       0       0       0     16.197458  

[604 rows x 138 columns]

### Functions to convert units, interacts with specific units dictionary above

In [18]:
def cup2oz(cup):
    cup=cup.astype('float32')
    oz=cup*8
    return(oz)

def gram2oz(gram):
    gram=gram.astype('float32')
    oz=gram/28.3495
    return(oz)

def T2oz(T):
    T=T.astype('float32')
    oz=T/2
    return(oz)

def egg2oz(egg):
    egg=egg.astype('float32')
    oz=egg*1.6 #about a large egg
    return(oz)

def serv2oz(serv):
    serv=serv.astype('float32')
    return(serv)

#### Check 

In [19]:
'DMF0100' in units['cup']['1'] 

True

## Convert all data

In [20]:
df09_conv=data_dict['set_09'][['DoB', 'Participant ID', 'Date of Intake','age at intake']+units['keep_serv']]
df04_conv=data_dict['set_04'][['DoB', 'Participant ID', 'Date of Intake','age at intake']+units['keep_04']]
df04_conv.drop_duplicates(['DoB', 'Participant ID', 'Date of Intake'])
for key, value in units.items():
    print(key)
    if key == 'cup':
        for unit, cols in value.items():
            df09_conv=df09_conv.join(cup2oz(data_dict['set_09'][cols])*float(unit))
    if key == 'TB':
        for unit, cols in value.items():
            df09_conv=df09_conv.join(T2oz(data_dict['set_09'][cols])*float(unit))
    if key == 'g':
        for unit, cols in value.items():
            df09_conv=df09_conv.join(gram2oz(data_dict['set_09'][cols])*float(unit))
    if key == 'egg':
        for unit, cols in value.items():
            df09_conv=df09_conv.join(egg2oz(data_dict['set_09'][cols])*float(unit))
    if key == 'oz':
        for unit, cols in value.items():
            df09_conv=df09_conv.join(serv2oz(data_dict['set_09'][cols])*float(unit))
    if key == 'floz':
        for unit, cols in value.items():
            df09_conv=df09_conv.join(serv2oz(data_dict['set_09'][cols])*float(unit))

keep_serv
keep_04
cup
floz
oz
TB
g
egg


### Merge the 09 and 04 datasets

In [21]:
df=df04_conv.merge(df09_conv, on=['DoB', 'Participant ID', 'Date of Intake','age at intake'],left_index=True, right_index=True)

### dv_conv is all in oz except(!) the sweets which are in "servings" as defined by NDSR
### Convert the all variables with hei to float

In [22]:
for key, values in hei_ped_dict['hei'].items():
    print(key)
    df[key]=df[values].astype('float32').sum(axis=1)

hei_sweets
hei_salty
hei_veg
hei_greensbeans
hei_totalfruit
hei_wholegrains
hei_refinedgrains
hei_proteins
hei_seafoodplantprot
hei_dairy
hei_dairyprot
hei_milk
hei_formula_foz
hei_cereal
hei_fruitjuice
hei_ssb
ENERGY


## Checking max values

In [23]:
frames=[]
for key, values in hei_ped_dict['hei'].items():
    frames.append(df.loc[df[[key]].idxmax()])

result = pd.concat(frames)
result['max value'] = list(hei_ped_dict['hei'].keys())


In [24]:
result

DoB  Participant ID Date of Intake  age at intake  \
22     6/9/15          423548     04/08/2018      33.971950   
468    9/4/16          518139     09/10/2018      24.181195   
65   11/21/15          960370     12/06/2017      24.509744   
183  12/28/16          645257     02/05/2018      13.273373   
36    6/19/15          320544     08/08/2016      13.667632   
541  12/19/16          515697     04/11/2018      15.704635   
450    2/4/16          488291     02/13/2018      24.312614   
101   5/11/16          277066     07/13/2017      14.061890   
183  12/28/16          645257     02/05/2018      13.273373   
394    9/8/16          107008     10/14/2017      13.174809   
375    4/8/17          240615     08/12/2018      16.131748   
401    9/8/16          107008     04/22/2017       7.425204   
585  12/10/17          911829     08/17/2018       8.213721   
392   4/29/15          648807     02/11/2018      33.479127   
343    8/3/16          798610     03/23/2018      19.614366   
403   10/5/15          157075     12/21/2017      26.546746   
271   6/15/17          684448     09/06/2018      14.718988   

     Whole Grains (ounce equivalents)  Sodium (mg)  \
22                              0.627     1407.318   
468                             0.725     2265.749   
65                              0.865      968.905   
183                             0.786      626.112   
36                              2.370     1005.901   
541                             7.778     2030.193   
450                             0.000     3364.253   
101                             0.283     3179.774   
183                             0.786      626.112   
394                             0.447      737.831   
375                             0.725     2136.513   
401                             0.000       79.618   
585                             0.000      293.435   
392                             1.813     1144.643   
343                             0.073      731.596   
403                             0.841     2970.841   
271                             1.925     4756.038   

     Refined Grains (ounce equivalents)  Added Sugars (by Total Sugars) (g)  \
22                                3.005                              29.800   
468                               6.715                              35.575   
65                                2.911                              14.746   
183                               2.319                               6.100   
36                                0.031                               8.692   
541                               3.115                              17.805   
450                               9.551                              30.343   
101                               2.075                              91.713   
183                               2.319                               6.100   
394                               0.039                              35.498   
375                               3.891                              18.244   
401                               0.375                               7.698   
585                               0.000                              70.476   
392                               3.746                              40.263   
343                               2.098                               5.449   
403                               4.032                              35.475   
271                               7.147                              31.930   

     % Calories from SFA  Energy (kcal)  ...  hei_seafoodplantprot  hei_dairy  \
22                14.673       1150.468  ...              0.000000   6.666000   
468                8.569       1561.518  ...              0.352500   1.630000   
65                 9.042        907.601  ...              2.400000   1.350000   
183                6.323        532.581  ...             18.029499  16.680000   
36                15.693       1054.579  ...              0.000000  12.466

## Sum the hei (low case) to the HEI variables. The HEI is the total amount of all the foods within each variable and will be then scored to a HEIX variable

In [25]:
for key, values in hei_ped_dict['HEI'].items():
    print(key)
    df[key]=df[values].astype('float32').sum(axis=1)


HEI_TOTALVEG
HEI_TOTALPROTEINS
HEI_MILK
HEI_TOTALFRUIT
HEI_WHOLEGRAINS
HEI_REFINEDGRAINS
HEI_FRUITJUICE
HEI_SSB
HEI_SWEETS
HEI_SALTY
HEI_CEREAL


In [26]:
df.iloc[:,165:-1].describe()

ENERGY  HEI_TOTALVEG  HEI_TOTALPROTEINS    HEI_MILK  \
count   604.000000    604.000000         604.000000  604.000000   
mean    742.456665      3.071524           3.104814   11.932439   
std     482.853088      4.425453           4.253373   10.297261   
min       3.500000      0.000000           0.000000    0.000000   
25%     404.085747      0.000000           0.000000    3.221250   
50%     705.753998      1.502000           1.694300   10.049000   
75%    1040.515503      4.489000           4.830000   18.000000   
max    2698.863037     40.799999          42.291000   53.183998   

       HEI_TOTALFRUIT  HEI_WHOLEGRAINS  HEI_REFINEDGRAINS  HEI_FRUITJUICE  \
count      604.000000       604.000000         604.000000      604.000000   
mean         4.636509         0.611343           1.537607        0.270040   
std          5.850755         0.897843           1.775105        1.587571   
min         -0.016000         0.000000           0.000000        0.000000   
25%          0.000000         0.000000           0.000000        0.000000   
50%          3.000000         0.274000           0.984500        0.000000   
75%          6.704000         0.897000           2.428500        0.000000   
max         44.868000         7.778000           9.551000       20.020000   

          HEI_SSB  HEI_SWEETS   HEI_SALTY  
count  604.000000  604.000000  604.000000  
mean     0.236887    0.428315    0.469692  
std      1.534358    0.955355    1.199039  
min      0.000000   -2.743000    0.000000  
25%      0.000000    0.000000    0.000000  
50%      0.000000    0.000000    0.000000  
75%      0.000000    0.505000    0.387000  
max     22.000000    8.100000   12.380000

## FInding unreasonable values

In [27]:
frames=[]
for key, values in hei_ped_dict['HEI'].items():
    frames.append(df.loc[df[[key]].idxmax()])

result2 = pd.concat(frames)
result2['max value'] = list(hei_ped_dict['HEI'].keys())


In [28]:
result2[hei_ped_dict['HEI']['HEI_TOTALPROTEINS']+['Participant ID']+['max value']]

hei_proteins  hei_seafoodplantprot  hei_dairyprot  Participant ID  \
183       18.8295             18.029499          5.432          645257   
183       18.8295             18.029499          5.432          645257   
595        0.0000              0.000000          0.000           51407   
36         1.6000              0.000000          3.466          320544   
541        3.0950              0.375000          0.000          515697   
450        6.4562              0.732000          0.470          488291   
343        0.1240              0.000000          0.266          798610   
403        3.4870              0.000000          0.708          157075   
22         0.0000              0.000000          2.666          423548   
468        2.3405              0.352500          1.630          518139   
392        1.0000              1.000000          0.666          648807   

             max value  
183       HEI_TOTALVEG  
183  HEI_TOTALPROTEINS  
595           HEI_MILK  
36      HEI_TOTALFRUIT  
541    HEI_WHOLEGRAINS  
450  HEI_REFINEDGRAINS  
343     HEI_FRUITJUICE  
403            HEI_SSB  
22          HEI_SWEETS  
468          HEI_SALTY  
392         HEI_CEREAL

In [29]:
result2[hei_ped_dict['hei']['hei_proteins']+['Participant ID']+['max value']]

MRF0100  MRL0100  MRF0200  MRL0200  MRF0300  MRL0300  MRF0400  MRL0400  \
183    0.000      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
183    0.000      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
595    0.000      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
36     0.000      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
541    0.000      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
450    0.617      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
343    0.000      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
403    0.000      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
22     0.000      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
468    0.988      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
392    0.000      0.0      0.0      0.0      0.0      0.0      0.0      0.0   

     MCF0200  MCL0200  ...  MOF0100  MOF0200  MOF0300  MOF0400  MOF0500  \
183      0.0    0.000  ...      0.0      0.0   0.8000      0.0   0.0295   
183      0.0    0.000  ...      0.0      0.0   0.8000      0.0   0.0295   
595      0.0    0.000  ...      0.0      0.0   0.0000      0.0   0.0000   
36       0.0    0.000  ...      0.0      0.0   1.6000      0.0   0.0000   
541      0.0    0.000  ...      0.0      0.0   1.6000      0.0   0.0000   
450      0.0    0.328  ...      0.0      0.0   1.7792      0.0   0.0000   
343      0.0    0.000  ...      0.0      0.0   0.0000      0.0   0.0000   
403      0.0    0.000  ...      0.0      0.0   0.0000      0.0   0.0000   
22       0.0    0.000  ...      0.0      0.0   0.0000      0.0   0.0000   
468      0.0    0.000  ...      0.0      0.0   0.0000      0.0   0.3525   
392      0.0    0.000  ...      0.0      0.0   0.0000      0.0   0.0000   

     MOF0600  MOF0700  VEG0700  Participant ID          max value  
183    0.000      0.0   18.000          645257       HEI_TOTALVEG  
183    0.000      0.0   18.000          645257  HEI_TOTALPROTEINS  
595    0.000      0.0    0.000           51407           HEI_MILK  
36     0.000      0.0    0.000          320544     HEI_TOTALFRUIT  
541    0.375      0.0    0.000          515697    HEI_WHOLEGRAINS  
450    0.000      0.0    0.732          488291  HEI_REFINEDGRAINS  
343    0.000      0.0    0.000          798610     HEI_FRUITJUICE  
403    0.000      0.0    0.000          157075            HEI_SSB  
22     0.000      0.0    0.000          423548         HEI_SWEETS  
468    0.000      0.0    0.000          518139          HEI_SALTY  
392    1.000      0.0    0.000          648807         HEI_CEREAL  

[11 rows x 29 columns]

# DQI scoring parameters
### About the DQI
from Rios et al. "Development of a Diet Quality Score for Infants and Toddlers and its association with weight"J Nutrit Health Food Sci. 2016 ; 4(4): .

## Diet Quality Index Scoring for infants 0–8 months
### This will should be the same as the ped08_dict

High Score is 55

| Component  | Scores  |
|---|---|
| Milk (type of feeding)  |   |
| Breastfeeding  | 15 (ex)  |
| Formula  |  10 (partial) |
|   |  5 (ex) |
| Cereal  | Not consumed = 5  |
|   | consumed = 0  |
| Protein  | Not consumed = 5  |
|   | consumed = 0  |
| Vegetables  | Not consumed = 5  |
|   | consumed = 0  |
| Fruits  | Not consumed = 5  |
|   | consumed = 0  |
| 100% fruit juice  | Not consumed = 5  |
|   | consumed = 0  |
| Sugar-sweetened beverages  | Not consumed = 5  |
|   | consumed = 0  |
| Sweets  | Not consumed = 5  |
|   | consumed = 0  |
| Salty snacks  | Not consumed = 5  |
|   | consumed = 0  |
| Total  | 0-55  |

## Diet Quality Index Scoring for infants 8-24 months
### This will should be the same as the ped811_dict and ped1124_dicts

| Component  | Score  | 8-11 months  | 12-24 months  |
|---|---|---|---|
| Milk  |   |   |   |
|  Breastfeeding | 10  | Yes  | Yes  |
| Formula/Cow's milk/non-dairy milks (fl oz/d) | 5  | 20-28  | 14-18  |
|   | 2.5  | 8.0–19.9 or 28.1–35.0  | 8.0–13.9 or 18.1–24.0  |
|   | 0  | <8 or >35  | <8.0 or >24.0  |
| Grains  |   |   |   |
|  Whole grains (oz/d) | 2.5  | 1.0–3.5  | 1.5–5.5  |
|   | 1.25  |  0.1–0.9 or 3.6–8.0 |  0.1–1.4 or 5.6–8.0 |
|   |  0 | 0 or >8.0  | 0 or >8.0  |
|  Refined grains (oz/d) | 2.5  |  0–1.5 |  0–1.8 |
|   |  1.25 |  1.6–3.5 |  1.9–4.2 |
|   | 0  | >3.5  | >4.2  |
| Proteins (oz/d) |  5 | 2.5–6.0  |  2.0–3.0 |
|   | 2.5  |  0.1–2.4 or 6.1–10.0 | 0.1–1.9 or 3.1–6.0  |
|   | 0  | 0 or >10  |  0 or >6.0 |
| Vegetables (oz/d) | 5  |  ≥2.0  | ≥8.0  |
|   | 2.5  |  0.1–1.9 | 0.1–7.9  |
|   |  0 |  0 |  0 |
| Fruits (oz/d) | 5  | ≥2.0  | ≥8.0  |
|   | 2.5  |  0.1–1.9 | 0.1–7.9  |
|   | 0  |  0 |  0 |
| 100% fruit juice (fl oz/d) | 5  |  0 |  0-4.0 |
|   | 2.5  | 0.1–6.0  | 4.1–6.0  |
|   |  0 | >6.0  | >6.0  |
| Sugar sweetened beverages (fl oz/d) | 5  |  0 | 0  |
|   | 2.5  | 0.1–4.0  | 4.1–6.0  |
|   |  0 | >6.0  | >6.0  |
| Sweets servings/d | 5  |  0 |  0 |
|   | 2.5  |  0.1-1.0 | 0.1-1.0  |
|   | 0  | >1.0  | >1.0  |
| Salty snacks oz/ds | 5  |  0 | 0  |
|   | 2.5  |  0.1-1.0 | 0.1-1.0  |
|   | 0  | >1.0  | >1.0  |


The scores were based on the age of introduction of these foods as recommended by the WIC program13, the WHO14 and the American Academy of Pediatrics15, not on portion sizes.


# Function HEI_components takes the dictionaries (ped811_dict etc) and does a boolean comparison on the datasets, the output is creating a new HEIX column per group (milk, vegetables, etc) with the score (0, 2.5, 5, 10, or 15)

# *This is where it is under estimating the scores* 
The boolean operators are not picking up the correct values. They just need tuning, speciifically the less than and greater than. 
### *The outputs should align with the tables above.*

In [30]:
def HEI_components(dictio, df):
    for key, value in dictio.items():
        print(key)
        if key in ['HEI_TOTALVEG','HEI_TOTALFRUIT']: 
            df.loc[(df[key] > value['parameters'][0] )& (df[key] <= value['parameters'][1]), value['name']] = 2.5
            df.loc[(df[key] >= value['parameters'][1]), value['name']] = 5
            df.loc[(df[key] < value['parameters'][0]), value['name']] = 0
        if key in ['HEI_FRUITJUICE','HEI_SSB','HEI_SWEETS','HEI_SALTY']: 
            df.loc[(df[key] > value['parameters'][0] )& (df[key] <= value['parameters'][1]), value['name']] = 2.5
            df.loc[(df[key] >= value['parameters'][1]), value['name']] = 0
            df.loc[(df[key] < value['parameters'][0]), value['name']] = 5
        if key in ['HEI_REFINEDGRAINS']: 
            df.loc[(df[key] > value['parameters'][0] )& (df[key] <= value['parameters'][1]), value['name']] = 1.25
            df.loc[(df[key] >= value['parameters'][1]), value['name']] = 0
            df.loc[(df[key] < value['parameters'][0]), value['name']] = 2.5
        if key in ['HEI_WHOLEGRAINS','HEI_MILK']: 
            print('low end %f'%value['parameters'][2])
            print('high end %f'%value['parameters'][3])
            df.loc[(df[key] >= value['parameters'][0]) & (df[key] <= value['parameters'][1]), value['name']] = 2.5
            df.loc[(df[key] > value['parameters'][3]) | (df[key] <= value['parameters'][2]) , value['name']] = 0
            df.loc[(df[key] > value['parameters'][2]) & (df[key] < value['parameters'][0]), value['name']] = 1.25
            df.loc[(df[key] > value['parameters'][1]) & (df[key] < value['parameters'][3]), value['name']] = 1.25
        if key in ['HEI_TOTALPROTEINS']: 
            print('low end %f'%value['parameters'][2])
            print('high end %f'%value['parameters'][3])
            df.loc[(df[key] >= value['parameters'][0]) & (df[key] <= value['parameters'][1]), value['name']] = 5
            df.loc[(df[key] > value['parameters'][3]) | (df[key] <= value['parameters'][2]) , value['name']] = 0
            df.loc[(df[key] > value['parameters'][2]) & (df[key] < value['parameters'][0]), value['name']] = 2.5
            df.loc[(df[key] > value['parameters'][1]) & (df[key] < value['parameters'][3]), value['name']] = 2.5
    return(df)

In [31]:
calc_dict={
    'infant':HEI_components(ped_dict['infant'],df.loc[df['age at intake'] <= 6]),
    'young':HEI_components(ped_dict['young'],df.loc[(df['age at intake'] > 6) & (df['age at intake'] <= 11)]),
    'child':HEI_components(ped_dict['child'],df.loc[(df['age at intake'] > 11 ) & (df['age at intake'] <= 24)])}

HEI_TOTALVEG
HEI_TOTALFRUIT
HEI_WHOLEGRAINS
low end 0.000000
high end 0.000000
HEI_MILK
low end 0.000000
high end 0.000000
HEI_TOTALPROTEINS
low end 0.000000
high end 0.000000
HEI_REFINEDGRAINS
HEI_FRUITJUICE
HEI_SSB
HEI_SWEETS
HEI_SALTY
HEI_CEREAL
HEI_TOTALVEG
HEI_TOTALFRUIT
HEI_WHOLEGRAINS
low end 0.000000
high end 8.000000
HEI_MILK
low end 8.000000
high end 35.000000
HEI_TOTALPROTEINS
low end 0.000000
high end 10.000000
HEI_REFINEDGRAINS
HEI_FRUITJUICE
HEI_SSB
HEI_SWEETS
HEI_SALTY
HEI_TOTALVEG
HEI_TOTALFRUIT
HEI_WHOLEGRAINS
low end 0.000000
high end 8.000000
HEI_MILK
low end 8.000000
high end 24.000000
HEI_TOTALPROTEINS
low end 0.000000
high end 6.000000
HEI_REFINEDGRAINS
HEI_FRUITJUICE
HEI_SSB
HEI_SWEETS
HEI_SALTY


In [32]:
# check shape
for k,v in calc_dict.items():
    print(v.shape)

(121, 187)
(154, 187)
(261, 187)


# Breastfeeding functions 
These are functions that take the data from the breastfeeding dataset and relate them to the diet dataset

In [33]:
def anyFORM(df):
    df.loc[(df['age_regular_formula'].isnull() == True), 'age_formula'] = df['age_any_formula']
    df.loc[(df['age_regular_formula'].isnull() == False), 'age_formula'] = df['age_regular_formula']

In [34]:
def BF_infant(df):
    df.loc[(df['breastfed'] == 'yes') & (df['age_stop'] > df['age at intake']) | (df['age_stop'].isnull() == True) & (df['age_formula'] > df['age at intake']) | df['age_formula'].isnull() == True, 'Infant'] = 'EXBF'
    df.loc[(df['breastfed'] == 'no') & (df['age_stop_formula'] > df['age at intake']), 'Infant'] = 'EXFORM'
    df.loc[(df['age_stop'] == 0) , 'Infant'] = 'EXFORM'
    df.loc[(df['breastfed'] == 'yes') & (df['age_formula'] < df['age at intake']), 'Infant'] = 'MIX'

In [35]:
def BF_young_child(df):
    df.loc[(df['breastfed'] == 'yes') & (df['age_stop'] >= df['age at intake']), 'child_young'] = 'BF'
    df.loc[(df['age_stop'] < df['age at intake']) | (df['age_stop'].isnull() == True), 'child_young'] = 'noBF'


In [36]:
def BF_HEI(df, age):
    if age == 'infant':
        df.loc[(df['Infant'] == 'EXBF'), 'HEIX0_BF'] = 15
        df.loc[(df['Infant'] == 'MIX'), 'HEIX0_BF'] = 10
        df.loc[(df['Infant'] == 'EXFORM'), 'HEIX0_BF'] = 5
    else:
        df.loc[(df['child_young'] == 'noBF'), 'HEIX0_BF'] = 0
        df.loc[(df['child_young'] == 'BF'), 'HEIX0_BF'] = 10


In [37]:
anyFORM(df_bf)

In [38]:
bf_dict = {}
for key, df in calc_dict.items():
    print(key)
    if key == 'infant':
        test = df.merge(df_bf, left_on='Participant ID', right_on='Participant ID')
        BF_infant(test)
        bf_dict[key] = test
        BF_HEI(test, key)
    else:
        test = df.merge(df_bf, left_on='Participant ID', right_on='Participant ID')
        BF_young_child(test)
        bf_dict[key] = test
        BF_HEI(test, key)

infant
young
child


In [39]:
HEI_cols = ['HEIX0_BF','HEIX1_TOTALVEGETABLES', 
            'HEIX2_TOTALFRUIT', 'HEIX3_WHOLEGRAIN', 
            'HEIX4_MILK', 'HEIX5_TOTALPROTEIN', 
            'HEIX6_REFINEDGRAIN', 'HEIX7_FRUITJUICE', 
            'HEIX8_SSB', 'HEIX9_SWEETS', 'HEIX10_SALTY']



### Function HEI_TOTAL is supposed to sum the columns of interest (HEI_cols) and produce the final score

In [40]:
def HEI_TOTAL(df):
    df['HEI_TOTAL']=df[HEI_cols].sum(axis=1)

In [43]:
bf_dict['infant']

DoB  Participant ID Date of Intake  age at intake  \
0    7/19/17            2081     08/12/2017       0.788517   
1    7/19/17            2081     08/03/2017       0.492823   
2    3/26/18           62519     04/11/2018       0.525678   
3    6/30/17          106436     07/21/2017       0.689953   
4    6/30/17          106436     07/26/2017       0.854227   
..       ...             ...            ...            ...   
113  4/25/17          981854     06/21/2017       1.872728   
114  4/25/17          993015     07/19/2017       2.792665   
115  4/25/17          993015     10/04/2017       5.322491   
116  4/25/17          993015     07/29/2017       3.121214   
117  4/25/17          993015     10/08/2017       5.453911   

     Whole Grains (ounce equivalents)  Sodium (mg)  \
0                               0.000      220.536   
1                               0.000      116.867   
2                               0.000        1.700   
3                               0.000        1.530   
4                               0.000        1.530   
..                                ...          ...   
113                             0.000        9.031   
114                             0.000      137.100   
115                             0.187      409.602   
116                             0.000      122.846   
117                             0.375      117.066   

     Refined Grains (ounce equivalents)  Added Sugars (by Total Sugars) (g)  \
0                                   0.0                              15.545   
1                                   0.0                              12.688   
2                                   0.0                               0.000   
3                                   0.0                               0.000   
4                                   0.0                               0.000   
..                                  ...                                 ...   
113                                 0.0                               0.000   
114                                 0.0                               0.000   
115                                 0.5                              33.102   
116                                 0.0                               0.000   
117                                 0.0                              26.173   

     % Calories from SFA  Energy (kcal)  ...  age_regular_formula  \
0                 21.028        532.440  ...                  NaN   
1                 19.224        357.150  ...                  NaN   
2                 25.227          7.000  ...                  NaN   
3                 25.227          6.300  ...                  1.0   
4                 25.227          6.300  ...                  1.0   
..                   ...            ...  ...                  ...   
113               25.227         37.188  ...                  NaN   
114               25.227        564.531  ...                  3.0   
115               11.221        625.722  ...                  3.0   
116               25.227        505.837  ...                  3.0   
117               14.771        401.814  ...                  3.0   

     age_stop_formula  age_fed    donor_milk donor_milk_percent  \
0                 NaN      5.0  not_answered                NaN   
1                 NaN      5.0  not_answered                NaN   
2                 NaN      6.0            no                NaN   
3                 5.0      6.0            no                NaN   
4                 5.0      6.0            no                NaN   
..                ...      ...           ...                ...   
113               NaN      4.0  not_answered                NaN   
114              12.0      4.0  not_answered                NaN   
115              12.0      4.0  not_answered                NaN   
116              12.0      4.0  not_answered                NaN   
117              12.0      4.0  not_answered                NaN   

          four_month_ratio age_f

In [41]:
HEI_TOTAL(bf_dict['infant'])

In [42]:
bf_dict['infant'].to_csv(os.path.join(basepath, 'final_dataout_infant.csv'))